<a href="https://colab.research.google.com/github/vu-bigdata-2021/homework-6/blob/master/emr-spark-wordcount-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install boto3 pyspark

In [ ]:
from google.colab import files

# upload AWS credentials
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [3]:
import boto3, json

with open('credentials.json') as f:
  credentials = json.load(f)

session = boto3.session.Session(**credentials)
s3 = session.client('s3')

In [4]:
# upload dataset to S3
s3.upload_file(Filename='sample_data/README.md', Bucket='vandy-bigdata', Key='datasets/README.md')

In [ ]:
%%file PysparkStreaming.py
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(appName="PysparkStreaming")
words = sc.textFile('s3://vandy-bigdata/datasets/README.md').flatMap(lambda line: line.split(' '))
counts = words.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("s3://vandy-bigdata/hw6/outputs/wordcount/")

In [6]:
# upload script to S3
s3.upload_file(Filename='PysparkStreaming.py', Bucket='vandy-bigdata', Key='hw6/PysparkStreaming.py')

In [ ]:
# submit the new spark streaming job
CLUSTER_ID = 'j-xxxxxxx'

emr = session.client('emr')
emr.add_job_flow_steps(JobFlowId=CLUSTER_ID, Steps=[{
    'Name': 'Spark Streaming',
    'ActionOnFailure': 'CANCEL_AND_WAIT',
    'HadoopJarStep': {
        'Args': ['spark-submit',
                 '--master', 'yarn',
                 '--deploy-mode', 'cluster',
                 's3://vandy-bigdata/hw6/PysparkStreaming.py'],
        'Jar': 'command-runner.jar'
    }}])